# Demo
@ Team 1

In [1]:
!git clone https://github.com/Hidden-Picture-Puzzle/Solving-Hidden-Picture-Puzzle.git
%cd Solving-Hidden-Picture-Puzzle
!ls

Cloning into 'Solving-Hidden-Picture-Puzzle'...
remote: Enumerating objects: 494, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 494 (delta 35), reused 4 (delta 4), pack-reused 449 (from 1)
Receiving objects: 100% (494/494), 47.18 MiB | 20.52 MiB/s, done.
Resolving deltas: 100% (97/97), done.
/content/Solving-Hidden-Picture-Puzzle
dataset  demo.ipynb  README.md	requirements.txt  result  test	test_result


In [2]:
!pip install -r requirements.txt

## Setting

In [3]:
import cv2
import numpy as np
import os
import glob
from scipy.ndimage import distance_transform_edt
from joblib import Parallel, delayed
import multiprocessing
import pandas as pd
import cv2.ximgproc

## Matching Algorithms

### Grayscale Chamfer Matching

In [4]:
class GrayscaleChamferMatcher:
    def __init__(self, scene_path: str, truncation_dist=40.0):
        self.scene = cv2.imread(scene_path, cv2.IMREAD_GRAYSCALE)
        if self.scene is None:
            raise ValueError(f"Failed to load scene: {scene_path}")

        self.truncation_dist = truncation_dist
        self.num_ori_bins = 8
        self.ori_bin_angle = 180.0 / self.num_ori_bins

        self.edges_fine, self.orient_map_fine = self._extract_edges_and_orientation_adaptive(self.scene)
        self.dir_dist_maps_fine = self._build_truncated_directional_distance_maps(
            self.edges_fine, self.orient_map_fine
        )
        self.integral_edges_fine = cv2.integral(self.edges_fine.astype(np.float32) / 255.0)

        self.coarse_scale = 0.5
        scene_small = cv2.resize(self.scene, None, fx=self.coarse_scale, fy=self.coarse_scale)

        self.edges_coarse, self.orient_map_coarse = self._extract_edges_and_orientation_adaptive(scene_small)
        self.dir_dist_maps_coarse = self._build_truncated_directional_distance_maps(
            self.edges_coarse, self.orient_map_coarse
        )
        self.integral_edges_coarse = cv2.integral(self.edges_coarse.astype(np.float32) / 255.0)

    def _extract_edges_and_orientation_adaptive(self, img):
        """
        Scene용: Canny(50,150) + Morph + Skeleton
        """
        blurred = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)

        edges = cv2.Canny(blurred, 50, 150, L2gradient=True)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel, iterations=1)

        num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(edges, connectivity=8)
        clean_edges = np.zeros_like(edges)
        for i in range(1, num_labels):
            if stats[i, cv2.CC_STAT_AREA] >= 10:
                clean_edges[labels == i] = 255

        skeleton = np.zeros(clean_edges.shape, np.uint8)
        img_temp = clean_edges.copy()
        element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))

        while True:
            eroded = cv2.erode(img_temp, element)
            temp = cv2.dilate(eroded, element)
            temp = cv2.subtract(img_temp, temp)
            skeleton = cv2.bitwise_or(skeleton, temp)
            img_temp = eroded.copy()
            if cv2.countNonZero(img_temp) == 0:
                break

        edges = skeleton

        gx = cv2.Sobel(img, cv2.CV_32F, 1, 0, ksize=3)
        gy = cv2.Sobel(img, cv2.CV_32F, 0, 1, ksize=3)
        grad_angle = np.rad2deg(np.arctan2(gy, gx))
        grad_angle[grad_angle < 0] += 180.0

        orient_map = np.full(grad_angle.shape, -1, dtype=np.float32)
        orient_map[edges > 0] = grad_angle[edges > 0]

        return edges, orient_map

    def _extract_template_edges_and_orientation(self, template_image):
        """
        Template용: Canny(30,100) + Skeleton (1px)
        """

        if len(template_image.shape) == 3:
            gray_img = cv2.cvtColor(template_image, cv2.COLOR_BGR2GRAY)
        else:
            gray_img = template_image

        blurred = cv2.bilateralFilter(gray_img, d=5, sigmaColor=30, sigmaSpace=30)
        edges = cv2.Canny(blurred, 30, 100, L2gradient=True)

        num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(edges, connectivity=8)
        clean_edges = np.zeros_like(edges)
        for i in range(1, num_labels):
            if stats[i, cv2.CC_STAT_AREA] >= 10:
                clean_edges[labels == i] = 255

        skeleton = np.zeros(clean_edges.shape, np.uint8)
        img_temp = clean_edges.copy()
        element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))

        while True:
            eroded = cv2.erode(img_temp, element)
            temp = cv2.dilate(eroded, element)
            temp = cv2.subtract(img_temp, temp)
            skeleton = cv2.bitwise_or(skeleton, temp)
            img_temp = eroded.copy()
            if cv2.countNonZero(img_temp) == 0:
                break

        edges = skeleton

        gx = cv2.Sobel(gray_img, cv2.CV_32F, 1, 0, ksize=3)
        gy = cv2.Sobel(gray_img, cv2.CV_32F, 0, 1, ksize=3)
        grad_angle = np.rad2deg(np.arctan2(gy, gx))
        grad_angle[grad_angle < 0] += 180.0

        orient_map = np.full(grad_angle.shape, -1, dtype=np.float32)
        orient_map[edges > 0] = grad_angle[edges > 0]

        return edges, orient_map

    def _build_truncated_directional_distance_maps(self, edges, orient_map):
        h, w = edges.shape
        dist_maps = []
        for b in range(self.num_ori_bins):
            ang_min = b * self.ori_bin_angle
            ang_max = (b + 1) * self.ori_bin_angle

            mask = np.zeros((h, w), dtype=np.uint8)
            bin_mask = (orient_map >= ang_min) & (orient_map < ang_max)
            mask[bin_mask] = 1

            inv_mask = 1 - mask
            dist = distance_transform_edt(inv_mask)

            # (최적화) Truncation 미리 적용
            dist = np.minimum(dist, self.truncation_dist).astype(np.float32)
            dist_maps.append(dist)

        return dist_maps

    def _rotate_template(self, image, angle):
        if angle == 0: return image.copy()
        (h, w) = image.shape[:2]
        (cX, cY) = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
        cos, sin = np.abs(M[0, 0]), np.abs(M[0, 1])
        nW = int((h * sin) + (w * cos))
        nH = int((h * cos) + (w * sin))
        M[0, 2] += (nW / 2) - cX
        M[1, 2] += (nH / 2) - cY
        return cv2.warpAffine(image, M, (nW, nH), flags=cv2.INTER_NEAREST, borderValue=0)

    def _get_template_masks_by_bin(self, template_gray):
        edges, grad_angle = self._extract_template_edges_and_orientation(template_gray)

        valid = edges > 0
        bins = (grad_angle[valid] // self.ori_bin_angle).astype(np.int32)
        bins = np.clip(bins, 0, self.num_ori_bins - 1)

        masks = []
        for b in range(self.num_ori_bins):
            mask = np.zeros_like(edges, dtype=np.float32)
            # bin_map을 다시 만드는 대신 바로 마스킹
            # 현재 bin인 픽셀들만 1.0
            mask_indices = (bins == b)
            # valid한 픽셀 중 bin이 b인 것들의 좌표
            y_idxs, x_idxs = np.where(valid)
            y_b = y_idxs[mask_indices]
            x_b = x_idxs[mask_indices]
            mask[y_b, x_b] = 1.0
            masks.append(mask)

        return masks, edges

    def _compute_score_map_fast(self, dist_maps, template_masks):
        total_cost_map = None

        for b in range(self.num_ori_bins):
            t_mask = template_masks[b]
            if np.sum(t_mask) == 0:
                continue

            # TM_CCORR: 템플릿(1)과 겹치는 이미지 픽셀(거리값)을 다 더함
            # 수식: sum( dist_map(x+i, y+j) * template(i, j) )
            cost = cv2.matchTemplate(dist_maps[b], t_mask, cv2.TM_CCORR)

            if total_cost_map is None:
                total_cost_map = cost
            else:
                total_cost_map += cost

        return total_cost_map

    def _get_edge_density_fast(self, integral_img, x, y, w, h):
        """Integral Image를 이용한 O(1) Edge Density 계산"""
        # Integral image는 원본보다 크기가 1씩 큼
        # Sum = I[y2, x2] - I[y1, x2] - I[y2, x1] + I[y1, x1]
        y2 = min(y + h, integral_img.shape[0] - 1)
        x2 = min(x + w, integral_img.shape[1] - 1)
        y1 = max(y, 0)
        x1 = max(x, 0)

        count = (integral_img[y2, x2]
                 - integral_img[y1, x2]
                 - integral_img[y2, x1]
                 + integral_img[y1, x1])
        return count

    def _process_single_scale_and_angle(self, scale, angle, template_gray_orig, orig_w, orig_h):
        # --- 1. Coarse 단계 (고속 필터링) ---
        c_w = int(orig_w * scale * self.coarse_scale)
        c_h = int(orig_h * scale * self.coarse_scale)

        # Scene(Coarse) 크기보다 템플릿이 크면 즉시 중단
        scene_h_coarse, scene_w_coarse = self.dir_dist_maps_coarse[0].shape

        if c_w >= scene_w_coarse or c_h >= scene_h_coarse: return None
        if c_w < 5 or c_h < 5: return None

        t_coarse_resized = cv2.resize(template_gray_orig, (c_w, c_h))
        t_coarse_rot = self._rotate_template(t_coarse_resized, angle)

        # 회전 후 크기가 Scene보다 큰지 한 번 더 체크
        tc_h, tc_w = t_coarse_rot.shape
        if tc_w > scene_w_coarse or tc_h > scene_h_coarse: return None

        t_coarse_masks, t_coarse_edges = self._get_template_masks_by_bin(t_coarse_rot)

        edge_pixel_count = np.sum(t_coarse_edges > 0)
        if edge_pixel_count == 0: return None

        # (최적화) 전체 맵에 대해 한 번에 점수 계산
        try:
            score_map_coarse = self._compute_score_map_fast(self.dir_dist_maps_coarse, t_coarse_masks)
        except cv2.error:
            return None

        if score_map_coarse is None: return None

        score_map_coarse /= edge_pixel_count

        # 최소 비용 위치 찾기
        min_val, _, min_loc, _ = cv2.minMaxLoc(score_map_coarse)

        # Coarse Threshold (truncation의 절반보다 크면 버림)
        if min_val > self.truncation_dist * 0.6:
            return None

        best_c_loc = min_loc  # (x, y)

        # --- 2. Fine 단계 (정밀 검사) ---
        # 좌표 복원
        orig_cx = int(best_c_loc[0] / self.coarse_scale)
        orig_cy = int(best_c_loc[1] / self.coarse_scale)

        f_w = int(orig_w * scale)
        f_h = int(orig_h * scale)

        scene_h_fine, scene_w_fine = self.dir_dist_maps_fine[0].shape
        if f_w >= scene_w_fine or f_h >= scene_h_fine: return None
        if f_w < 5 or f_h < 5: return None

        t_fine_resized = cv2.resize(template_gray_orig, (f_w, f_h))
        t_fine_rot = self._rotate_template(t_fine_resized, angle)

        tf_h, tf_w = t_fine_rot.shape
        if tf_w > scene_w_fine or tf_h > scene_h_fine: return None

        t_fine_masks, t_fine_edges = self._get_template_masks_by_bin(t_fine_rot)

        fine_edge_count = np.sum(t_fine_edges > 0)
        if fine_edge_count == 0: return None

        # ROI 설정
        margin = 20
        start_x = max(0, orig_cx - margin)
        start_y = max(0, orig_cy - margin)
        end_x = min(self.dir_dist_maps_fine[0].shape[1], orig_cx + f_w + margin)
        end_y = min(self.dir_dist_maps_fine[0].shape[0], orig_cy + f_h + margin)

        roi_w = end_x - start_x
        roi_h = end_y - start_y

        # ROI가 템플릿보다 작으면 매칭 불가
        if roi_w < tf_w or roi_h < tf_h: return None

        # ROI 크롭된 Distance Maps
        roi_dist_maps = [d[start_y:end_y, start_x:end_x] for d in self.dir_dist_maps_fine]

        # ROI 내에서 매칭
        try:
            score_map_fine = self._compute_score_map_fast(roi_dist_maps, t_fine_masks)
        except cv2.error:
            return None

        if score_map_fine is None: return None

        score_map_fine /= fine_edge_count

        # ROI 내 최소값
        min_dist, _, min_loc_roi, _ = cv2.minMaxLoc(score_map_fine)

        # 전체 좌표계로 변환
        final_x = start_x + min_loc_roi[0]
        final_y = start_y + min_loc_roi[1]

        # Edge Density Penalty (O(1) 조회)
        scene_edge_cnt = self._get_edge_density_fast(
            self.integral_edges_fine, final_x, final_y, tf_w, tf_h
        )
        ratio = scene_edge_cnt / fine_edge_count
        penalty = 1.0
        if ratio > 6.0: # max_ratio
            penalty = ratio / 6.0

        final_dist = min_dist * penalty

        # Score 변환 (0~1)
        final_score = max(0, 1.0 - (final_dist / self.truncation_dist))

        return {
            'scale': round(scale, 2),
            'angle': angle,
            'score': round(final_score, 4),
            'chamfer_distance': round(final_dist, 4),
            'location': (final_x, final_y),
            'template_size': (f_w, f_h)
        }

    def match(self, template_path, label, scale_range=(0.7, 1.3), scale_step=0.1, rotation_step=15, **kwargs):
        # 템플릿 로드 확인
        template_gray_orig = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
        if template_gray_orig is None:
            print(f"Error: Cannot read template {template_path}")
            return {'label': label, 'best_score': 0, 'top_results': []}

        orig_h, orig_w = template_gray_orig.shape

        scales = np.arange(scale_range[0], scale_range[1] + scale_step / 2, scale_step)
        angles = np.arange(0, 360, rotation_step)

        # 병렬 처리
        results = Parallel(n_jobs=-1)(
            delayed(self._process_single_scale_and_angle)(scale, angle, template_gray_orig, orig_w, orig_h)
            for scale in scales
            for angle in angles
        )

        valid_results = [r for r in results if r is not None]
        top_results = sorted(valid_results, key=lambda x: x['score'], reverse=True)[:3]

        if not top_results:
            return {'label': label, 'best_score': 0, 'best_scale': 1.0, 'best_angle': 0, 'best_location': (0,0), 'top_results': []}

        best_res = top_results[0]
        print(f"   --> Best: Scale {best_res['scale']}x | Angle {best_res['angle']}° | Score {best_res['score']:.4f}")

        return {
            'label': label,
            'best_scale': best_res['scale'],
            'best_angle': best_res['angle'],
            'best_score': best_res['score'],
            'best_location': best_res['location'],
            'best_template_size': best_res['template_size'],
            'top_results': top_results
        }

### HSV 필터링 헬퍼 함수

In [5]:
def find_optimal_k(data, max_k=8, threshold=0.15):
    n_samples = data.shape[0]
    if n_samples < max_k: return max(1, n_samples)

    prev_compactness = float('inf')
    best_k = 1

    for k in range(1, max_k + 1):
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
        compactness, _, _ = cv2.kmeans(data, k, None, criteria, 3, cv2.KMEANS_RANDOM_CENTERS)

        if k == 1:
            prev_compactness = compactness
            continue

        improvement = (prev_compactness - compactness) / prev_compactness
        if improvement < threshold:
            best_k = k - 1
            break
        best_k = k
        prev_compactness = compactness

    return best_k

def get_hsv_ranges(template_path,
                   auto_k=True, k=5,
                   margin_h=10, margin_s=30, margin_v=30,
                   min_valid_pixels=50):

    img_bgr = cv2.imread(template_path)
    if img_bgr is None:
        print(f"오류: {template_path} 파일을 읽을 수 없습니다.")
        return []

    # --- 1. GrabCut: 배경(흰색, 박스 등)을 날리고 물체만 남김 ---
    mask = np.zeros(img_bgr.shape[:2], np.uint8)
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)

    h, w = img_bgr.shape[:2]
    rect = (1, 1, w-2, h-2)

    # GrabCut 실행 (반복 5회)
    try:
        cv2.grabCut(img_bgr, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)
    except Exception as e:
        print(f"GrabCut 실패: {e}")
        return []

    # 0(배경), 2(배경추정) -> 0 / 1(전경), 3(전경추정) -> 1
    mask_final = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

    # 물체 픽셀 개수 확인
    if np.count_nonzero(mask_final) < min_valid_pixels:
        print("GrabCut 결과 물체가 감지되지 않았습니다.")
        return []

    # --- 2. 물체 영역의 HSV 픽셀만 추출 ---
    img_hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)

    # 마스크가 1인 부분(물체)의 픽셀만 가져옴
    valid_data = img_hsv[mask_final == 1].astype(np.float32)

    s_median = np.median(valid_data[:, 1])
    if s_median < 8:
        print("Template grayscale -> skip HSV")
        return []

    # --- 3. K-Means 군집화 및 범위 산출 ---
    final_k = k
    if auto_k:
        sample_data = valid_data
        if len(valid_data) > 1000:
            indices = np.random.choice(len(valid_data), 1000, replace=False)
            sample_data = valid_data[indices]
        final_k = find_optimal_k(sample_data, max_k=8)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    _, labels, centers = cv2.kmeans(valid_data, final_k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    ranges = []

    for i in range(final_k):
        # 현재 그룹의 픽셀들
        cluster_pixels = valid_data[labels.flatten() == i]

        if len(cluster_pixels) < min_valid_pixels // final_k:
            continue

        h_vals, s_vals, v_vals = cluster_pixels[:, 0], cluster_pixels[:, 1], cluster_pixels[:, 2]

        # S, V 범위 계산
        s_min = max(0, np.percentile(s_vals, 5) - margin_s)
        s_max = min(255, np.percentile(s_vals, 95) + margin_s)
        v_min = max(0, np.percentile(v_vals, 5) - margin_v)
        v_max = min(255, np.percentile(v_vals, 95) + margin_v)

        # Hue Wrap-around 처리 (빨간색 0/179 경계 문제 해결을 위해)
        h_shifted = (h_vals + 90) % 180
        h_start_s, h_end_s = np.percentile(h_shifted, 5), np.percentile(h_shifted, 95)
        h_start_s -= margin_h
        h_end_s += margin_h

        h_start = (h_start_s - 90) % 180
        h_end = (h_end_s - 90) % 180

        # 경계선 침범 체크
        if h_start_s < 0 or h_end_s > 179 or h_start > h_end:
            # 범위가 끊어진 경우 -> 두 개로 분할
            ranges.append(((int(h_start), int(s_min), int(v_min)), (179, int(s_max), int(v_max))))
            ranges.append(((0, int(s_min), int(v_min)), (int(h_end), int(s_max), int(v_max))))
        else:
            # 일반적인 경우 -> 하나로 추가
            ranges.append(((int(h_start), int(s_min), int(v_min)), (int(h_end), int(s_max), int(v_max))))

    return ranges

def is_scene_grayscale(scene_bgr, s_thresh=12, gray_ratio=0.95):
    """
    scene이 대부분 무채색이면 True
    - HSV에서 S(채도)가 매우 낮은 픽셀 비율이 gray_ratio 이상이면 흑백/무채색으로 판단
    """
    hsv = cv2.cvtColor(scene_bgr, cv2.COLOR_BGR2HSV)
    s = hsv[:, :, 1]
    ratio = np.mean(s < s_thresh)
    return ratio >= gray_ratio

def create_color_mask(scene_path, template_path):
    scene_bgr = cv2.imread(scene_path)
    if scene_bgr is None:
        raise FileNotFoundError(f"Scene 이미지를 찾을 수 없습니다: {scene_path}")

    if is_scene_grayscale(scene_bgr):
        return None

    hsv_scene = cv2.cvtColor(scene_bgr, cv2.COLOR_BGR2HSV)

    ranges = get_hsv_ranges(template_path, auto_k=True)

    if not ranges:
        return None

    final_mask = np.zeros(hsv_scene.shape[:2], dtype=np.uint8)
    for lower, upper in ranges:
        mask = cv2.inRange(hsv_scene, np.array(lower, dtype=np.uint8), np.array(upper, dtype=np.uint8))
        final_mask = cv2.bitwise_or(final_mask, mask)

    return final_mask

### Color GHT Matching

In [6]:
class ColorGHTMatcher:
    def __init__(self, scene_path: str):
        self.scene_bgr = cv2.imread(scene_path)
        self.scene = cv2.cvtColor(self.scene_bgr, cv2.COLOR_BGR2GRAY)

        # 노이즈 제거
        self.scene = cv2.bilateralFilter(self.scene, 5, 50, 50)

        # 1. Fine Scale (원본)
        self.grad_fine, self.orient_fine, self.mask_fine = self._compute_gradients(self.scene)

        # 2. Coarse Scale (1/2 축소)
        self.coarse_scale = 0.5
        scene_small = cv2.resize(self.scene, None, fx=self.coarse_scale, fy=self.coarse_scale)
        self.grad_coarse, self.orient_coarse, self.mask_coarse = self._compute_gradients(scene_small)

    def _compute_gradients(self, img):
        """Sobel Gradient, Orientation, Edge Mask 계산"""
        dx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
        dy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)

        magnitude = cv2.magnitude(dx, dy)
        orientation = np.degrees(np.arctan2(dy, dx)).astype(int) % 360

        # 엣지 마스크 (너무 약한 엣지는 투표에 참여 안 시킴)
        mask = magnitude > 30
        return magnitude, orientation, mask

    def _build_r_table(self, template):
        """Template의 기본 R-Table 생성 (0도, 1.0배)"""
        dx = cv2.Sobel(template, cv2.CV_64F, 1, 0, ksize=3)
        dy = cv2.Sobel(template, cv2.CV_64F, 0, 1, ksize=3)

        magnitude = cv2.magnitude(dx, dy)
        orientation = np.degrees(np.arctan2(dy, dx)).astype(int) % 360
        edge_mask = magnitude > 40

        h, w = template.shape
        xc, yc = w // 2, h // 2

        y_idxs, x_idxs = np.where(edge_mask)
        mags = magnitude[edge_mask]
        ors = orientation[edge_mask]

        r_table = {}
        for x, y, mag, angle in zip(x_idxs, y_idxs, mags, ors):
            if angle not in r_table: r_table[angle] = []
            # (dx, dy, mag) 저장
            r_table[angle].append((xc - x, yc - y, mag))

        return r_table, (w, h)

    def _get_rotated_r_table(self, r_table, angle, scale):
        """
        R-table을 미리 회전 및 스케일링하여 새로운 테이블 생성
        -> Inner Loop에서 sin/cos 연산 제거 -> 속도 향상
        """
        theta_rad = np.radians(angle)
        cos_t = np.cos(theta_rad)
        sin_t = np.sin(theta_rad)

        rotated_r_table = {}

        for t_angle, vectors in r_table.items():
            # 템플릿이 회전하면 엣지 각도도 회전함
            # Scene에서 찾을 때는 (기존 각도 + 회전 각도)를 키로 사용
            scene_query_angle = int(t_angle + angle) % 360

            new_vectors = []
            for dx, dy, mag in vectors:
                # 1. 벡터 회전
                rdx = dx * cos_t - dy * sin_t
                rdy = dx * sin_t + dy * cos_t

                # 2. 스케일 적용
                final_dx = int(rdx * scale)
                final_dy = int(rdy * scale)

                new_vectors.append((final_dx, final_dy, mag))

            if scene_query_angle not in rotated_r_table:
                rotated_r_table[scene_query_angle] = []
            rotated_r_table[scene_query_angle].extend(new_vectors)

        return rotated_r_table

    def _vote(self, rotated_r_table, scene_grad, scene_orient, scene_mask):
        h, w = scene_grad.shape
        accumulator = np.zeros((h, w), dtype=np.float32)

        # Scene 엣지 픽셀 추출
        y_idxs, x_idxs = np.where(scene_mask)
        s_mags = scene_grad[scene_mask]
        s_ors = scene_orient[scene_mask]

        # 투표
        for i, (sy, sx) in enumerate(zip(y_idxs, x_idxs)):
            scene_angle = s_ors[i]

            # 미리 만들어둔 Rotated Table 조회
            if scene_angle in rotated_r_table:
                for rdx, rdy, rmag in rotated_r_table[scene_angle]:
                    # 이미 회전/스케일링 된 벡터를 더하기만 함
                    xc = sx + rdx
                    yc = sy + rdy

                    if 0 <= xc < w and 0 <= yc < h:
                        # Weighted Voting
                        accumulator[yc, xc] += (s_mags[i] * rmag)

        # Smoothing
        accumulator = cv2.GaussianBlur(accumulator, (9, 9), 2.0)
        return accumulator

    def _process_coarse_to_fine(self, scale, angle, r_table, t_w, t_h, color_mask=None):
        # --- 1. Coarse Search (1/2 Scale) ---
        coarse_eff_scale = scale * self.coarse_scale

        # Coarse 레벨 마스크 적용
        final_mask_coarse = self.mask_coarse
        if color_mask is not None:
            mask_small = cv2.resize(color_mask, None, fx=self.coarse_scale, fy=self.coarse_scale, interpolation=cv2.INTER_NEAREST)
            # "엣지도 있고(True) AND 색깔도 맞는(>0) 곳"만 남김
            final_mask_coarse = self.mask_coarse & (mask_small > 0)

        # 엣지가 하나도 없으면 스킵 (속도 향상)
        if not np.any(final_mask_coarse):
            return None

        coarse_r_table = self._get_rotated_r_table(r_table, angle, coarse_eff_scale)

        # 투표 함수에 '합쳐진 마스크'를 전달
        acc_coarse = self._vote(
            coarse_r_table,
            self.grad_coarse, self.orient_coarse, final_mask_coarse
        )

        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(acc_coarse)
        if max_val < 500: return None

        # Coarse 위치 -> Fine 위치 변환
        center_x_coarse, center_y_coarse = max_loc
        orig_cx = int(center_x_coarse / self.coarse_scale)
        orig_cy = int(center_y_coarse / self.coarse_scale)

        # --- 2. Fine Search (ROI Only) ---
        margin = 30
        fine_h, fine_w = self.grad_fine.shape

        start_x = max(0, orig_cx - margin - int(t_w*scale/2))
        start_y = max(0, orig_cy - margin - int(t_h*scale/2))
        end_x = min(fine_w, orig_cx + margin + int(t_w*scale/2))
        end_y = min(fine_h, orig_cy + margin + int(t_h*scale/2))

        if end_x <= start_x or end_y <= start_y: return None

        # ROI 잘라내기
        grad_roi = self.grad_fine[start_y:end_y, start_x:end_x]
        orient_roi = self.orient_fine[start_y:end_y, start_x:end_x]
        mask_roi = self.mask_fine[start_y:end_y, start_x:end_x]

        # Fine 레벨 마스크 적용 (ROI 부분만 잘라서)
        if color_mask is not None:
            color_mask_roi = color_mask[start_y:end_y, start_x:end_x]
            mask_roi = mask_roi & (color_mask_roi > 0)

        if not np.any(mask_roi): return None

        fine_r_table = self._get_rotated_r_table(r_table, angle, scale)
        acc_fine_roi = self._vote(fine_r_table, grad_roi, orient_roi, mask_roi)

        f_min, f_max, f_loc_min, f_loc_max = cv2.minMaxLoc(acc_fine_roi)

        final_cx = start_x + f_loc_max[0]
        final_cy = start_y + f_loc_max[1]

        return {
            'scale': scale,
            'angle': angle,
            'votes': f_max,
            'location': (final_cx, final_cy),
            'box': (int(final_cx - t_w*scale//2), int(final_cy - t_h*scale//2),
                    int(t_w*scale), int(t_h*scale))
        }

    def match(self, template_path, label, scale_range=(0.8, 1.4), scale_step=0.1, rotation_step=15, color_mask=None):
        template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
        r_table, (t_w, t_h) = self._build_r_table(template)

        print(f"\n{'='*90}")
        print(f"   Optimized GHT (Rotated Table + Coarse-to-Fine): {label}")
        print(f"   Scale: {scale_range} | Rot Step: {rotation_step}°")

        scales = np.arange(scale_range[0], scale_range[1] + 1e-5, scale_step)
        angles = np.arange(0, 360, rotation_step)

        results = Parallel(n_jobs=-1)(
            delayed(self._process_coarse_to_fine)(scale, angle, r_table, t_w, t_h, color_mask)
            for scale in scales
            for angle in angles
        )

        valid_results = [r for r in results if r is not None]
        valid_results.sort(key=lambda x: x['votes'], reverse=True)

        top_results = valid_results[:3]

        if not top_results:
             print("   ❌ No match found.")
             return {'label': label, 'best_score': 0, 'best_scale': 1.0, 'best_location': (0,0), 'top_results': []}

        best_res = top_results[0]
        norm_score = min(1.0, best_res['votes'] / 100000.0)

        print(f"\n상위 결과:")
        for i, res in enumerate(top_results, 1):
            print(f"  {i}. Scale {res['scale']:.2f}x | Angle {res['angle']}°| Votes {best_res['votes']:.0f}")

        center_x, center_y = best_res['location']
        bw, bh = best_res['box'][2], best_res['box'][3]

        return {
            'label': label,
            'best_scale': best_res['scale'],
            'best_angle': best_res['angle'],
            'best_score': norm_score,
            'best_votes': best_res['votes'],
            'best_location': (int(center_x - bw/2), int(center_y - bh/2)),
            'best_template_size': (bw, bh),
            'top_results': valid_results[:3]
        }

### Wrapper

이미지 타입에 따라 자동으로 적절한 매처(Matcher)를 선택하는 Wrapper Class

In [7]:
class MultiMatcher:
    """
    이미지 타입(컬러/흑백)을 감지하여 적절한 Matcher로 연결하는 Wrapper 클래스
    """
    is_wrapper = True

    def __init__(self, scene_path: str, **kwargs):
        self.scene_path = scene_path

        test_img = cv2.imread(scene_path, cv2.IMREAD_COLOR)

        if test_img is None:
            raise ValueError(f"Cannot load scene image: {scene_path}")

        is_grayscale = False

        if len(test_img.shape) == 2 or (len(test_img.shape) == 3 and test_img.shape[2] == 1):
            is_grayscale = True
        else:
            hsv = cv2.cvtColor(test_img, cv2.COLOR_BGR2HSV)
            saturation = hsv[:, :, 1]
            mean_saturation = np.mean(saturation)

            if mean_saturation < 4.0:
                is_grayscale = True
                print(f"[Check] 3-channel image but Low Saturation (Mean S={mean_saturation:.2f}). Treating as Grayscale.")

        if is_grayscale:
            print(f"🖤 [Info] Grayscale Scene detected ({scene_path}). Using Grayscale Matcher.")
            self.matcher = GrayscaleChamferMatcher(scene_path, **kwargs)
        else:
            print(f"🎨 [Info] Color Scene detected ({scene_path}). Using Original Color Matcher.")
            self.matcher = ColorGHTMatcher(scene_path)

    def match(self, *args, **kwargs):
        return self.matcher.match(*args, **kwargs)

    def __getattr__(self, name):
        return getattr(self.matcher, name)

## Detection

In [8]:
def find_all_objects(scene_path: str,
                     template_paths: dict,
                     method: str = 'multi',
                     scale_range=(0.8, 1.4),
                     scale_step=0.1,
                     **kwargs):

    if method == 'multi':
        matcher = MultiMatcher(scene_path, **kwargs)
        match_func = matcher.match
    else:
        raise ValueError(f"Unknown method: {method}")

    all_detections = []

    print(f"Finding All Objects using {method.upper()} method")
    print(f"{'='*90}\n")

    for label, template_path in template_paths.items():
        print(f"\n🔍 Searching for: {label}")

        result = match_func(
            template_path=template_path,
            label=label,
            scale_range=scale_range,
            scale_step=scale_step
        )

        all_detections.append(result)

        score = result['best_score']
        scale = result['best_scale']
        angle = result.get('best_angle')

        if angle is not None:
            print(f"   ✓ Best score: {score:.4f} at scale {scale:.2f}x, angle {angle}°")
        else:
            print(f"   ✓ Best score: {score:.4f} at scale {scale:.2f}x")

    return all_detections

## Visualization

In [9]:
def visualize_all_objects(scene_path: str,
                         detections: list,
                         output_path: str,
                         method_name: str = "Multi-Object Detection",
                         show_scores: bool = True):
    # Scene 로드
    scene = cv2.imread(scene_path)
    if scene is None:
        raise ValueError(f"Cannot load scene image: {scene_path}")

    scene_h, scene_w = scene.shape[:2]

    # Scene 파일명 추출 (파일명에 포함시킬 용도)
    scene_name = os.path.splitext(os.path.basename(scene_path))[0]

    # 색상 팔레트
    colors = [
        (0, 255, 0),      # Green
        (255, 0, 0),      # Blue
        (0, 165, 255),    # Orange
        (255, 0, 255),    # Magenta
        (0, 255, 255),    # Yellow
        (255, 255, 0),    # Cyan
        (128, 0, 128),    # Purple
        (255, 128, 0),    # Sky Blue
    ]

    # 각 감지 결과 그리기
    for idx, result in enumerate(detections):
        label = result['label']
        location = result['best_location']
        template_size = result['best_template_size']
        score = result['best_score']
        scale = result['best_scale']
        angle = result.get('best_angle', 0) # angle 정보가 없으면 0으로 처리

        x, y = int(location[0]), int(location[1])
        w, h = int(template_size[0]), int(template_size[1])

        color = colors[idx % len(colors)]

        # 바운딩 박스 그리기
        if x >= 0 and y >= 0 and x + w <= scene_w and y + h <= scene_h:
            cv2.rectangle(scene, (x, y), (x + w, y + h), color, 2)

            # 라벨 텍스트 준비
            if show_scores:
                angle_text = f", {angle}°" if angle != 0 else ""
                text = f"{label} ({scale:.1f}x{angle_text}, {score:.2f})"
            else:
                text = f"{label}"

            # 텍스트 배경
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.4
            thickness = 2

            text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]

            # 텍스트 위치
            text_x = x
            text_y = y - 5 if y - 5 > 15 else y + h + 15

            # 텍스트 배경 그리기
            cv2.rectangle(scene,
                         (text_x, text_y - text_size[1] - 4),
                         (text_x + text_size[0] + 4, text_y + 4),
                         color, -1)

            # 텍스트 그리기
            cv2.putText(scene, text, (text_x + 2, text_y),
                       font, font_scale, (255, 255, 255), thickness)

    # 메서드 제목
    title = f"{method_name} - {len(detections)} objects detected - Scene: {scene_name}"
    title_font_scale = 0.6
    title_thickness = 2

    title_size = cv2.getTextSize(title, cv2.FONT_HERSHEY_SIMPLEX, title_font_scale, title_thickness)[0]

    # 제목 배경
    cv2.rectangle(scene, (5, 5), (15 + title_size[0], 15 + title_size[1]), (0, 0, 0), -1)
    # 제목 텍스트
    cv2.putText(scene, title, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, title_font_scale, (0, 255, 255), title_thickness)

    # 결과 저장
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cv2.imwrite(output_path, scene)
    print()
    print(f"✓ Visualization saved: {output_path}")

    return scene

## Evaluation

In [10]:
def auto_find_templates(scene_path, template_dir):
    """
    Scene 파일 경로와 템플릿 디렉토리에서 자동으로 템플릿 찾기
    """
    # Scene 파일명 추출
    scene_filename = os.path.basename(scene_path)
    scene_prefix = os.path.splitext(scene_filename)[0]

    # 템플릿 디렉토리에서 해당 scene의 템플릿 찾기
    template_paths = {}

    # 패턴: scene_prefix-*.jpg
    pattern = os.path.join(template_dir, f'{scene_prefix}-*.*')

    for filepath in glob.glob(pattern):
        filename = os.path.basename(filepath)

        # 확장자가 이미지 파일인지 확인
        if not filename.lower().endswith(('.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG')):
            continue

        # 라벨 추출
        name_without_ext = os.path.splitext(filename)[0]
        label = name_without_ext.replace(f'{scene_prefix}-', '', 1)

        template_paths[label] = filepath

    return template_paths

In [11]:
def compare_all_methods(scene_path: str,
                       template_paths: dict,
                       output_dir: str,
                       scale_range=(0.5, 2.0),
                       scale_step=0.2,
                       methods=('multi',)):
    """
    여러 방법 비교
    (최종적으로 multi(Chamfer for Grayscale, GHT for Color) method만 이용함)
    """
    scene_name = os.path.splitext(os.path.basename(scene_path))[0]
    all_results = {}

    for method in methods:
        detections = find_all_objects(
            scene_path=scene_path,
            template_paths=template_paths,
            method=method,
            scale_range=scale_range,
            scale_step=scale_step
        )

        all_results[method] = detections

        output_path = os.path.join(output_dir, f"{scene_name}_all_objects_{method}.png")
        visualize_all_objects(
            scene_path=scene_path,
            detections=detections,
            output_path=output_path,
            method_name=f"{method.upper()} Matching"
        )

    # methods 개수에 맞춰 비교 이미지 생성 (methods가 1개면 비교 이미지도 1개 폭)
    scene = cv2.imread(scene_path)
    scene_h, scene_w = scene.shape[:2]

    comparison = np.zeros((scene_h, scene_w * len(methods), 3), dtype=np.uint8)

    for col_idx, method in enumerate(methods):
        result_path = os.path.join(output_dir, f"{scene_name}_all_objects_{method}.png")
        result_img = cv2.imread(result_path)
        if result_img is None:
            continue
        comparison[:, col_idx*scene_w:(col_idx+1)*scene_w] = result_img

    comparison_path = os.path.join(output_dir, f"{scene_name}_comparison_all_methods.png")
    cv2.imwrite(comparison_path, comparison)

    return all_results

In [12]:
possible_exts = ["png", "PNG", "jpg", "JPG", "jpeg", "JPEG"]

def find_template_file(scene_id, template_dir, template_name):
    base = f"{scene_id}-{template_name}"
    for ext in possible_exts:
        fp = os.path.join(template_dir, f"{base}.{ext}")
        if os.path.exists(fp):
            return fp
    return None

def get_template_radius(scene_id, template_dir, template_name):
    file_path = find_template_file(scene_id, template_dir, template_name)
    if file_path is None or (not os.path.exists(file_path)):
        print(f"[경고] 템플릿 이미지 없음: {scene_id}-{template_name} → 기본 반지름 40")
        return 40

    img = cv2.imread(file_path)
    if img is None:
        print(f"[경고] 템플릿 읽기 실패: {file_path} → 기본 반지름 40")
        return 40

    h, w = img.shape[:2]
    radius = int(max(w, h) * 0.5)
    radius = min(radius, 120)
    return radius

def calculate_iou(boxA, boxB):
    xA, yA, wA, hA = boxA
    xB, yB, wB, hB = boxB
    A = (xA, yA, xA + wA, yA + hA)
    B = (xB, yB, xB + wB, yB + hB)

    x1 = max(A[0], B[0]); y1 = max(A[1], B[1])
    x2 = min(A[2], B[2]); y2 = min(A[3], B[3])

    iw = max(0, x2 - x1)
    ih = max(0, y2 - y1)
    inter = iw * ih

    areaA = wA * hA
    areaB = wB * hB
    denom = (areaA + areaB - inter)
    if denom <= 0:
        return 0.0
    return inter / denom

In [13]:
def draw_dashed_circle(img, center, radius, color, thickness=2, dash_len=10):
    x, y = center
    for angle in range(0, 360, dash_len * 2):
        cv2.ellipse(img, (x, y), (radius, radius), 0, angle, angle + dash_len, color, thickness)

def visualize_combined_results(scene_path: str,
                               detections: list,
                               df_scene_gt: pd.DataFrame,
                               output_path: str,
                               template_dir: str,
                               scene_id: str,
                               iou_threshold: float = 0.5):
    scene = cv2.imread(scene_path)
    if scene is None:
        raise ValueError(f"Cannot load scene image: {scene_path}")

    scene_h, scene_w = scene.shape[:2]

    COLOR_TP = (255, 0, 0)       # Blue
    COLOR_FP = (0, 0, 255)       # Red
    COLOR_FN = (100, 100, 100)   # Dark Gray

    matched_gt_indices = set()


# 1. Detection 순회 및 매칭 확인
    for det in detections:
        label = det['label']
        x, y = map(int, det['best_location'])
        w, h = map(int, det['best_template_size'])

        # 중심점 계산
        det_cx, det_cy = x + w / 2, y + h / 2

        # 해당 라벨의 GT 후보군 추출
        gt_candidates = df_scene_gt[df_scene_gt['template'] == label]

        is_correct = False

        # GT 후보들과 거리 비교
        for idx, row in gt_candidates.iterrows():
            gt_x, gt_y = int(row['x']), int(row['y'])
            radius = get_template_radius(scene_id, template_dir, label)

            # 거리 계산
            dist = ((det_cx - gt_x)**2 + (det_cy - gt_y)**2)**0.5

            # 정답 판별 (0.5R 이내)
            if dist <= radius * 0.5:
                is_correct = True
                matched_gt_indices.add(idx)

        if is_correct:
            # True Positive (Blue Box)
            cv2.rectangle(scene, (x, y), (x + w, y + h), COLOR_TP, 3)
            cv2.putText(scene, f"{label}", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, COLOR_TP, 2)
        else:
            # False Positive (Red Box w/ X mark)
            cv2.line(scene, (x, y), (x + w, y + h), COLOR_FP, 3)
            cv2.line(scene, (x + w, y), (x, y + h), COLOR_FP, 3)
            cv2.putText(scene, f"{label}", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, COLOR_FP, 2)

    # 2. 미검출 GT (False Negative) 순회
    # matched_gt_indices에 없는 GT만 회색 점선으로 그림
    for idx, row in df_scene_gt.iterrows():
        if idx not in matched_gt_indices:
            gt_x, gt_y = int(row['x']), int(row['y'])
            template_name = row['template']
            radius = get_template_radius(scene_id, template_dir, template_name)

            draw_dashed_circle(scene, (gt_x, gt_y), radius, COLOR_FN, thickness=2, dash_len=10)
            cv2.putText(scene, f"{template_name}", (gt_x - radius, gt_y - radius - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_FN, 2)

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cv2.imwrite(output_path, scene)
    print(f"✓ Visualization saved: {output_path}")
    return scene

def parse_scene_gt(df_gt_all, scene_id):
    df_scene = df_gt_all[df_gt_all['scene'] == scene_id].copy()
    if df_scene.empty:
        return df_scene

    split_coords = df_scene['px'].astype(str).str.split(',', expand=True)
    df_scene['x'] = split_coords[0].str.strip().astype(int)
    df_scene['y'] = split_coords[1].str.strip().astype(int)
    return df_scene

def evaluate_one_scene(scene_path, template_dir, output_dir,
                       df_gt_all=None, df_scene_gt=None, scale_range=(0.7, 1.0), scale_step=0.1,
                       iou_threshold=0.5):
    scene_id = os.path.splitext(os.path.basename(scene_path))[0]
    scene_out_dir = os.path.join(output_dir, scene_id)
    os.makedirs(scene_out_dir, exist_ok=True)

    # 템플릿 자동 탐색
    template_paths = auto_find_templates(scene_path, template_dir)
    if not template_paths:
        print(f"⚠️ [{scene_id}] 템플릿이 없어 스킵합니다.")
        return None

    # 탐지 실행 (현재 ght만)
    results = compare_all_methods(
        scene_path=scene_path,
        template_paths=template_paths,
        output_dir=scene_out_dir,
        scale_range=scale_range,
        scale_step=scale_step,
        methods=('multi',)
    )

    detections = results['multi']  # list

    # GT 없으면 탐지만 저장하고 종료
    if df_scene_gt is None or df_scene_gt.empty:
        print(f"⚠️ [{scene_id}] GT가 없어 평가 스킵 (탐지만 저장).")
        return {
            'scene_id': scene_id,
            'n_templates': len(template_paths),
            'accuracy': None,
            'matched': '',
            'unmatched': '',
            'metrics_csv': None,
            '_correct_cnt': 0,
            '_total_cnt': 0
        }

    df_scene = df_scene_gt

    # Distance/IoU 계산
    distances = []
    ious = []

    for det in detections:
        label = det['label']
        x_det, y_det = det['best_location']
        w_det, h_det = det['best_template_size']

        cx_det = x_det + w_det / 2
        cy_det = y_det + h_det / 2

        gt_entry = df_scene[df_scene['template'] == label]
        if gt_entry.empty:
            continue

        cx_gt = int(gt_entry['x'].iloc[0])
        cy_gt = int(gt_entry['y'].iloc[0])

        dist = float(np.sqrt((cx_det - cx_gt) ** 2 + (cy_det - cy_gt) ** 2))
        distances.append((label, dist))

        # IoU (GT는 원->bbox 근사)
        radius = get_template_radius(scene_id, template_dir, label)
        gt_bbox = (cx_gt - radius, cy_gt - radius, 2 * radius, 2 * radius)
        det_box = (int(x_det), int(y_det), int(w_det), int(h_det))
        iou = float(calculate_iou(det_box, gt_bbox))
        ious.append((label, iou))

    # 결과 DF
    dist_map = {k: v for k, v in distances}
    iou_map = {k: v for k, v in ious}

    final_rows = []
    for det in detections:
        label = det['label']
        gt_entry = df_scene[df_scene['template'] == label]
        if gt_entry.empty:
            gt_px = "N/A"
        else:
            gt_px = f"{int(gt_entry['x'].iloc[0])}, {int(gt_entry['y'].iloc[0])}"

        final_rows.append({
            'scene_id': scene_id,
            'template_name': label,
            'ground_truth_px': gt_px,
            'distance': f"{dist_map.get(label, 'N/A'):.2f}" if label in dist_map else "N/A",
            'iou': f"{iou_map.get(label, 'N/A'):.4f}" if label in iou_map else "N/A"
        })

    results_df = pd.DataFrame(final_rows)

    metrics_csv_path = os.path.join(scene_out_dir, f"{scene_id}_detection_metrics.csv")
    results_df.to_csv(metrics_csv_path, index=False)
    print(f"✓ Detection metrics saved to: {metrics_csv_path}")

    # Combined 시각화
    combined_output_path = os.path.join(scene_out_dir, f"{scene_id}_det_and_gt_combined.png")
    visualize_combined_results(
        scene_path=scene_path,
        detections=detections,
        df_scene_gt=df_scene,
        output_path=combined_output_path,
        template_dir=template_dir,
        scene_id=scene_id,
        iou_threshold=iou_threshold
    )

    # Accuracy (0.5R)
    correct, wrong = [], []
    for _, row in results_df.iterrows():
        name = row['template_name']
        dist_val = row['distance']

        distance = float(dist_val) if dist_val != "N/A" else float('inf')
        radius = get_template_radius(scene_id, template_dir, name)

        if distance <= (radius * 0.5):
            correct.append(name)
        else:
            wrong.append(name)

    total = len(results_df)
    acc = (len(correct) / total) * 100 if total > 0 else 0.0

    print(f"[{scene_id}] Accuracy(<=0.5R): {acc:.2f}% | Matched: {len(correct)}/{total}")

    return {
        'scene_id': scene_id,
        'n_templates': len(template_paths),
        'accuracy': acc,
        'matched': ', '.join(correct) if correct else '',
        'unmatched': ', '.join(wrong) if wrong else '',
        'metrics_csv': metrics_csv_path,
        '_correct_cnt': len(correct),
        '_total_cnt': total
    }

In [14]:
def get_df_scene_gt(scene_id: str, gt_dir: str, gt_cache: dict):
    prefix = scene_id[:2].lower()
    gt_csv_path = os.path.join(gt_dir, f"{prefix}-ground-truth.csv")

    if prefix not in gt_cache:
        if not os.path.exists(gt_csv_path):
            print(f"⚠️ GT CSV 없음: {gt_csv_path}")
            gt_cache[prefix] = pd.DataFrame(columns=['scene','template','px'])
        else:
            gt_cache[prefix] = pd.read_csv(gt_csv_path)

    df_all = gt_cache[prefix]
    if df_all.empty:
        return df_all.copy()

    # CSV의 scene 컬럼과 비교
    df_scene = df_all[df_all['scene'] == scene_id].copy()
    if df_scene.empty:
        return df_scene

    # px "378, 50" -> x,y
    split_coords = df_scene['px'].astype(str).str.split(',', expand=True)
    df_scene['x'] = split_coords[0].str.strip().astype(int)
    df_scene['y'] = split_coords[1].str.strip().astype(int)
    return df_scene

In [15]:
def run_all_scenes_batch(scenes_dir, template_dir, output_dir,
                         gt_dir=None, scale_range=(0.7, 1.0), scale_step=0.1,
                         iou_threshold=0.5):

    scene_files = []
    scene_files = sorted(
        glob.glob(os.path.join(scenes_dir, '*.jpg')) +
        glob.glob(os.path.join(scenes_dir, '*.JPG')) +
        glob.glob(os.path.join(scenes_dir, '*.png')) +
        glob.glob(os.path.join(scenes_dir, '*.PNG')) +
        glob.glob(os.path.join(scenes_dir, '*.jpeg')) +
        glob.glob(os.path.join(scenes_dir, '*.JPEG'))
    )

    scene_files = scene_files[:]
    print("✅ Run:", [os.path.basename(p) for p in scene_files])

    os.makedirs(output_dir, exist_ok=True)

    # prefix별 GT csv 캐시
    gt_cache = {}

    summary_rows = []

    # 전체 정확도 계산용 누적 카운터 (배치 단위)
    total_correct = 0
    total_total = 0

    for scene_path in scene_files:
        scene_id = os.path.splitext(os.path.basename(scene_path))[0]
        print(f"\n{'='*90}\n🚀 Processing: {scene_id}\n{'='*90}")

        # scene_id prefix에 맞는 GT 로드
        df_scene_gt = None
        if gt_dir is not None:
            df_scene_gt = get_df_scene_gt(scene_id, gt_dir, gt_cache)

        try:
            info = evaluate_one_scene(
                scene_path=scene_path,
                template_dir=template_dir,
                output_dir=output_dir,
                df_scene_gt=df_scene_gt,
                scale_range=scale_range,
                scale_step=scale_step,
                iou_threshold=iou_threshold
            )

            if info is None:
                continue

            # 배치 전체 정확도용 누적
            total_correct += info.get('_correct_cnt', 0)
            total_total   += info.get('_total_cnt', 0)

            # summary row (total_accuracy는 나중에 한 번에 넣음)
            summary_rows.append({
                'scene_id': info['scene_id'],
                'n_templates': info['n_templates'],
                'accuracy': info['accuracy'],
                'matched': info['matched'],
                'unmatched': info['unmatched'],
                'metrics_csv': info['metrics_csv'],
            })

        except Exception as e:
            print(f"❌ [{scene_id}] 처리 중 오류: {e}")
            summary_rows.append({
                'scene_id': scene_id,
                'n_templates': None,
                'accuracy': None,
                'matched': '',
                'unmatched': '',
                'metrics_csv': None,
            })

    # 배치 전체 정확도계산
    total_acc = (total_correct / total_total * 100) if total_total > 0 else None

    summary_df = pd.DataFrame(summary_rows)

    # 모든 행에 동일한 total_accuracy 넣기
    summary_df['total_accuracy'] = total_acc

    # 컬럼 순서 정렬
    summary_df = summary_df[['scene_id', 'n_templates', 'accuracy', 'matched', 'unmatched', 'total_accuracy', 'metrics_csv']]

    summary_csv = os.path.join(output_dir, "batch_summary.csv")
    summary_df.to_csv(summary_csv, index=False)
    print(f"\n✅ Batch summary saved: {summary_csv}")
    print(f"✅ Total Accuracy (all scenes): {total_acc if total_acc is not None else 'N/A'}")

    return summary_df

## Main

In [16]:
scenes_dir = '/content/Solving-Hidden-Picture-Puzzle/test'
template_dir = '/content/Solving-Hidden-Picture-Puzzle/dataset/templates'
gt_dir = '/content/Solving-Hidden-Picture-Puzzle/dataset/ground-truth'
output_dir = '/content/Solving-Hidden-Picture-Puzzle/test_result'

RANGE = (0.7, 1.3)
STEP = 0.1

summary_df = run_all_scenes_batch(
    scenes_dir=scenes_dir,
    template_dir=template_dir,
    output_dir=output_dir,
    gt_dir=gt_dir,
    scale_range=RANGE,
    scale_step=STEP,
    iou_threshold=0.5
)

display(summary_df)

✅ Run: ['ce-14.JPG', 'ge-07.jpg']

🚀 Processing: ce-14
🎨 [Info] Color Scene detected (/content/Solving-Hidden-Picture-Puzzle/test/ce-14.JPG). Using Original Color Matcher.
Finding All Objects using MULTI method


🔍 Searching for: fried chicken

   Optimized GHT (Rotated Table + Coarse-to-Fine): fried chicken
   Scale: (0.7, 1.3) | Rot Step: 15°


KeyboardInterrupt: 